In [ ]:
%pylab inline

import pandas as pd
import pickle
import matplotlib.dates as md
import statsmodels.api as sm
import scipy.stats as dists

from scipy.misc import factorial
from scipy.optimize import minimize

tableau = pickle.load(open('tableau.colors', 'rb'))

rcParams['figure.figsize'] = (10,8)
rcParams['axes.grid'] = True
rcParams['lines.linewidth'] = 2.5
rcParams['axes.prop_cycle'] = cycler('color', tableau)

xfmt = md.DateFormatter('%H:%M:%S')

Populating the interactive namespace from numpy and matplotlib


In [ ]:
hardcoded = pd.read_csv('~/Downloads/coinbaseUSD.csv', header=0, names=['t', 'px', 'sz'])

In [ ]:
df = hardcoded.copy()
df.t *= 1e9
df.t = pd.to_datetime(df.t)
df['pxd'] = df.px - df.px.shift(1)
df['d'] = 1*(df.pxd > 0) - 1*(df.pxd < 0)

# Eliminate early data with insufficient liquidity.
df = df.loc[df.t > "2015-09-03"]

In [4]:
def front(df, stamp, days=1):
    sliced = df.loc[(df.t - pd.to_datetime(stamp)) >= pd.Timedelta(days=0)]
    sliced = sliced.loc[(sliced.t - pd.to_datetime(stamp)) < pd.Timedelta(days=days)]
    return sliced

def around(df, stamp, size):
    sliced = df.loc[abs(df.t - pd.to_datetime(stamp)) < pd.Timedelta(days=size/24.)]
    return sliced

def vwap(df, window):
    return (df.px * df.sz).rolling(window).sum() / df.sz.rolling(window).sum()

In [5]:
unique_days = df.t.map(lambda t: t.strftime('%Y-%m-%d')).unique()

In [71]:
# PIN Model.
# Params: alpha, theta, mu, epsilon_buy, epsilon_sell
def log_likelihood(buys, sells, xk):
    alpha, theta, mu, epsilon_buy, epsilon_sell = xk
    k1 = -mu - buys*log(1 + mu/epsilon_buy)
    k2 = -mu - sells*log(1 + mu/epsilon_sell)
    k3 = -buys*log(1 + mu/epsilon_buy) - sells*log(1 + mu/epsilon_sell)
    kmax = max(k1, k2, k3)
    
    a = log(alpha*theta*exp(k1-kmax) + alpha*(1-theta)*exp(k2-kmax) + (1-alpha)*exp(k3-kmax))
    b = buys*log(epsilon_buy + mu) + sells*log(epsilon_sell + mu)
    return a + b + (epsilon_buy + epsilon_sell) + kmax

def pin(xk):
    alpha, theta, mu, epsilon_buy, epsilon_sell = xk
    return alpha*mu/(epsilon_buy + epsilon_sell + alpha*mu)

def initial_values(mbuys, msells):
    values = [0.1, 0.3, 0.5, 0.7, 0.9]
    choices = []
    for alpha in values:
        for theta in values:
            for gamma in values:
                if mbuys <= msells:
                    epsilon_buy = gamma*mbuys
                    mu = (mbuys - epsilon_buy)/(alpha*(1-theta))
                    epsilon_sell = msells - alpha*theta*mu
                else:
                    epsilon_sell = gamma*msells
                    mu = (msells - epsilon_sell)/(alpha*theta)
                    epsilon_buy = mbuys - alpha*(1-theta)*mu
                if epsilon_buy > 0 and epsilon_sell > 0 and mu > 0:
                    choices.append((alpha, theta, mu, epsilon_buy, epsilon_sell))
    return choices

In [129]:
data = list(map(lambda day: front(df, day), unique_days[-100:]))
buys = array(list(map(lambda day: len(day.loc[day.d == 1]), data)))
sells = array(list(map(lambda day: len(day.loc[day.d == -1]), data)))

In [132]:
def obj(xk):
    return -sum(list(map(lambda idx: log_likelihood(buys[idx], sells[idx], xk), range(len(buys)))))

cons = [{'type':'ineq', 'fun':lambda xk: xk[0]},
        {'type':'ineq', 'fun':lambda xk: xk[1]},
        {'type':'ineq', 'fun':lambda xk: 1-xk[0]},
        {'type':'ineq', 'fun':lambda xk: 1-xk[1]}]

choices = initial_values(mean(buys), mean(sells))

results = map(lambda choice: minimize(obj, choice, constraints=cons), choices)
#results = [res for res in results if res['message'] == 'Optimization terminated successfully.']
results = [res for res in results if (res['x'][0] > 1e-4) and (res['x'][1] > 1e-4) and (res['x'][2] > 0)]
optimum = min(results, key=lambda res: -obj(res['x']))
optimum

     fun: -9363295468.1479053
     jac: array([ 256.,    0.,    0.,    0.,    0.,    0.])
 message: 'Optimization terminated successfully.'
    nfev: 32
     nit: 3
    njev: 3
  status: 0
 success: True
       x: array([  6.45280398e-01,   1.51321912e-04,   3.83902555e+06,
         4.64325040e+07,   4.69605832e+07])

In [133]:
pin(optimum['x'])

0.025839566823874213

In [134]:
optimum['x'][3]/(optimum['x'][3]+optimum['x'][4])

0.49717281418461756